Import des librairies

In [1]:
from cryptography.hazmat.primitives.twofactor import hotp
from cryptography.hazmat.primitives import twofactor, hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import base64

# Génération des 3 premières valeurs de HOTP en fonction de la clé choisie

In [2]:
key = "3B7KHDWFFZ5LXPEEKQQEF7TB56GUPQHQ"
keyBytes = base64.b32decode(key)

myHotp = hotp.HOTP(keyBytes, 6, hashes.SHA1())

for count in range(10):
    print(f"{count} value: {myHotp.generate(count)}")

0 value: b'722129'
1 value: b'014956'
2 value: b'548211'
3 value: b'662234'
4 value: b'430129'
5 value: b'433270'
6 value: b'529782'
7 value: b'692621'
8 value: b'252337'
9 value: b'466414'


In [3]:
# Test HOTP invalide
try:
    myHotp.verify(b"123456", 0)
except twofactor.InvalidToken:
    print(f"123456 invalidToken")

123456 invalidToken


# Génération d'une clé ECDH

In [33]:
alicePvtKey = X25519PrivateKey.generate()
alicePubKey = alicePvtKey.public_key()

serverPvtKey = X25519PrivateKey.generate()
serverPubKey = serverPvtKey.public_key()

aliceSharedKey = alicePvtKey.exchange(serverPubKey)
aliceDerivedKey = HKDF(
    algorithm=hashes.SHA256(),
    length=20,
    salt=None,
    info=b'Alice&Server PSK',
).derive(aliceSharedKey)
serverSharedKey = serverPvtKey.exchange(alicePubKey)
serverDerivedKey = HKDF(
    algorithm=hashes.SHA256(),
    length=20,
    salt=None,
    info=b'Alice&Server PSK',
).derive(aliceSharedKey)



In [38]:
aliceHotp = hotp.HOTP(aliceDerivedKey, 6, hashes.SHA1())
serverHotp = hotp.HOTP(serverDerivedKey, 6, hashes.SHA1())

for count in range(10):
    print(f"{count} =>\tAlice value: {aliceHotp.generate(count)}\tServer value: {serverHotp.generate(count)}")

0 =>	Alice value: b'119733'	Server value: b'119733'
1 =>	Alice value: b'200416'	Server value: b'200416'
2 =>	Alice value: b'415773'	Server value: b'415773'
3 =>	Alice value: b'959557'	Server value: b'959557'
4 =>	Alice value: b'512617'	Server value: b'512617'
5 =>	Alice value: b'042536'	Server value: b'042536'
6 =>	Alice value: b'452491'	Server value: b'452491'
7 =>	Alice value: b'179756'	Server value: b'179756'
8 =>	Alice value: b'797101'	Server value: b'797101'
9 =>	Alice value: b'975404'	Server value: b'975404'
